In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from string import punctuation
import csv
import pandas as pd

### Preprocess the media dataset

In [2]:
media = pd.read_json('media.jsonl')

In [3]:
media.head()

,entities,created_at,reply_settings,conversation_id,possibly_sensitive,text,source,id,public_metrics,context_annotations,author_id,lang,author,__twarc,referenced_tweets,attachments,in_reply_to_user,in_reply_to_user_id
0,"{'annotations': [{'start': 31, 'end': 33, 'pro...",2021-12-31 12:06:05,everyone,1476887381336801280,False,'That was not the basis of the CDC's decision'...,SocialFlow,1476887381336801280,"{'retweet_count': 21, 'reply_count': 83, 'like...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN
1,"{'annotations': [{'start': 31, 'end': 33, 'pro...",2021-12-30 02:46:02,everyone,1476384051154112512,False,'That was not the basis of the CDC's decision'...,SocialFlow,1476384051154112512,"{'retweet_count': 25, 'reply_count': 54, 'like...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN
2,"{'annotations': [{'start': 31, 'end': 33, 'pro...",2021-12-29 13:31:07,everyone,1476184003526938624,False,'That was not the basis of the CDC's decision'...,SocialFlow,1476184003526938624,"{'retweet_count': 31, 'reply_count': 61, 'like...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN
3,"{'annotations': [{'start': 31, 'end': 33, 'pro...",2021-12-29 06:31:02,everyone,1476078288661581824,False,'That was not the basis of the CDC's decision'...,SocialFlow,1476078288661581824,"{'retweet_count': 15, 'reply_count': 34, 'like...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN
4,"{'annotations': [{'start': 0, 'end': 18, 'prob...",2021-08-25 19:34:01,everyone,1430614459077349376,False,Delta Air Lines CEO tells employees that they ...,SocialFlow,1430614459077349376,"{'retweet_count': 104, 'reply_count': 90, 'lik...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN


In [4]:
len(media)

42042

In case any tweets are retrieved more than once, drop duplicates based on the tweet id

In [5]:
media = media.drop_duplicates(subset=['id'], ignore_index = True)

In [6]:
len(media)

38184

Add name of the authors to the dataframe

In [7]:
m_author = pd.json_normalize(media["author"])
m_author.head()

,protected,name,id,description,username,url,verified,profile_image_url,created_at,entities.url.urls,entities.description.mentions,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,location,pinned_tweet_id,entities.description.urls,entities.description.hashtags
0,False,MSNBC,2836421,"The place for in-depth analysis, political com...",MSNBC,https://t.co/YBwtJIhwY3,True,https://pbs.twimg.com/profile_images/132108981...,2007-03-29T13:15:41.000Z,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 88, 'end': 99, 'username': 'MSNBCDa...",4695613,790,278871,0,NaN,NaN,NaN,NaN
1,False,MSNBC,2836421,"The place for in-depth analysis, political com...",MSNBC,https://t.co/YBwtJIhwY3,True,https://pbs.twimg.com/profile_images/132108981...,2007-03-29T13:15:41.000Z,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 88, 'end': 99, 'username': 'MSNBCDa...",4695613,790,278871,0,NaN,NaN,NaN,NaN
2,False,MSNBC,2836421,"The place for in-depth analysis, political com...",MSNBC,https://t.co/YBwtJIhwY3,True,https://pbs.twimg.com/profile_images/132108981...,2007-03-29T13:15:41.000Z,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 88, 'end': 99, 'username': 'MSNBCDa...",4695613,790,278871,0,NaN,NaN,NaN,NaN
3,False,MSNBC,2836421,"The place for in-depth analysis, political com...",MSNBC,https://t.co/YBwtJIhwY3,True,https://pbs.twimg.com/profile_images/132108981...,2007-03-29T13:15:41.000Z,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 88, 'end': 99, 'username': 'MSNBCDa...",4695613,790,278871,0,NaN,NaN,NaN,NaN
4,False,MSNBC,2836421,"The place for in-depth analysis, political com...",MSNBC,https://t.co/YBwtJIhwY3,True,https://pbs.twimg.com/profile_images/132108981...,2007-03-29T13:15:41.000Z,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 88, 'end': 99, 'username': 'MSNBCDa...",4695613,790,278871,0,NaN,NaN,NaN,NaN


In [8]:
media = media.join(m_author["username"])

In [9]:
media.head()

,entities,created_at,reply_settings,conversation_id,possibly_sensitive,text,source,id,public_metrics,context_annotations,author_id,lang,author,__twarc,referenced_tweets,attachments,in_reply_to_user,in_reply_to_user_id,username
0,"{'annotations': [{'start': 31, 'end': 33, 'pro...",2021-12-31 12:06:05,everyone,1476887381336801280,False,'That was not the basis of the CDC's decision'...,SocialFlow,1476887381336801280,"{'retweet_count': 21, 'reply_count': 83, 'like...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN,MSNBC
1,"{'annotations': [{'start': 31, 'end': 33, 'pro...",2021-12-30 02:46:02,everyone,1476384051154112512,False,'That was not the basis of the CDC's decision'...,SocialFlow,1476384051154112512,"{'retweet_count': 25, 'reply_count': 54, 'like...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN,MSNBC
2,"{'annotations': [{'start': 31, 'end': 33, 'pro...",2021-12-29 13:31:07,everyone,1476184003526938624,False,'That was not the basis of the CDC's decision'...,SocialFlow,1476184003526938624,"{'retweet_count': 31, 'reply_count': 61, 'like...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN,MSNBC
3,"{'annotations': [{'start': 31, 'end': 33, 'pro...",2021-12-29 06:31:02,everyone,1476078288661581824,False,'That was not the basis of the CDC's decision'...,SocialFlow,1476078288661581824,"{'retweet_count': 15, 'reply_count': 34, 'like...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN,MSNBC
4,"{'annotations': [{'start': 0, 'end': 18, 'prob...",2021-08-25 19:34:01,everyone,1430614459077349376,False,Delta Air Lines CEO tells employees that they ...,SocialFlow,1430614459077349376,"{'retweet_count': 104, 'reply_count': 90, 'lik...","[{'domain': {'id': '45', 'name': 'Brand Vertic...",2836421,en,"{'protected': False, 'name': 'MSNBC', 'entitie...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,None,NaN,MSNBC


Minimize the dataset, keep only the columns needed for the study

In [10]:
media_min = media[['created_at','text','id', 'author_id', 'lang', 'username']]
media_min.head()

,created_at,text,id,author_id,lang,username
0,2021-12-31 12:06:05,'That was not the basis of the CDC's decision'...,1476887381336801280,2836421,en,MSNBC
1,2021-12-30 02:46:02,'That was not the basis of the CDC's decision'...,1476384051154112512,2836421,en,MSNBC
2,2021-12-29 13:31:07,'That was not the basis of the CDC's decision'...,1476184003526938624,2836421,en,MSNBC
3,2021-12-29 06:31:02,'That was not the basis of the CDC's decision'...,1476078288661581824,2836421,en,MSNBC
4,2021-08-25 19:34:01,Delta Air Lines CEO tells employees that they ...,1430614459077349376,2836421,en,MSNBC


Check missing values

In [11]:
media_min.isna().sum()

created_at    0
text          0
id            0
author_id     0
lang          0
username      0
dtype: int64

Check language, to make it easier for human interpretation, only keep the English tweets

In [12]:
media_min["lang"].value_counts()

en     38138
es        12
und       10
it         8
fr         6
de         3
ja         1
fi         1
in         1
ht         1
hu         1
ca         1
sv         1
Name: lang, dtype: int64

In [13]:
media_min = media_min[media_min['lang']=='en']

In [14]:
len(media_min)

38138

Some typical clean-up steps

In [15]:
media_text = media_min["text"].to_list()

In [16]:
media_text[:20]

["'That was not the basis of the CDC's decision': Dr. Fauci discusses Delta Air Lines CEO level of involvement in new CDC isolation guidelines for Covid-19\nhttps://t.co/SG4aprCgfg",
 "'That was not the basis of the CDC's decision': Dr. Fauci discusses Delta Air Lines CEO level of involvement in new CDC isolation guidelines for Covid-19\nhttps://t.co/4URxTOtiDS",
 "'That was not the basis of the CDC's decision': Dr. Fauci discusses Delta Air Lines CEO level of involvement in new CDC isolation guidelines for Covid-19\nhttps://t.co/rNzOGIygXE",
 "'That was not the basis of the CDC's decision': Dr. Fauci discusses Delta Air Lines CEO level of involvement in new CDC isolation guidelines for Covid-19\nhttps://t.co/Pjmkqtm9AY",
 'Delta Air Lines CEO tells employees that they will face $200 monthly increases on their health insurance premiums starting Nov. 1 if they aren’t vaccinated against Covid-19. https://t.co/Yx3pKsQlkf',
 'More than 400 companies — including Tesla, Pfizer, Delta Air Lin

In [17]:
media_text = [re.sub(r'https?://\S+', '', text) for text in media_text] #remove url
media_text = ["".join([l for l in text if l not in punctuation]) for text in media_text]  #remove punctuation
media_text = [text.lower() for text in media_text]  # convert to lower case
media_text = [text.replace ("rt", "") for text in media_text] # remove the word "rt"
media_text = [" ".join(text.split()) for text in media_text]   # remove double spaces by splitting the strings into words and joining these words again

In [18]:
media_text[:20]

['that was not the basis of the cdcs decision dr fauci discusses delta air lines ceo level of involvement in new cdc isolation guidelines for covid19',
 'that was not the basis of the cdcs decision dr fauci discusses delta air lines ceo level of involvement in new cdc isolation guidelines for covid19',
 'that was not the basis of the cdcs decision dr fauci discusses delta air lines ceo level of involvement in new cdc isolation guidelines for covid19',
 'that was not the basis of the cdcs decision dr fauci discusses delta air lines ceo level of involvement in new cdc isolation guidelines for covid19',
 'delta air lines ceo tells employees that they will face 200 monthly increases on their health insurance premiums staing nov 1 if they aren’t vaccinated against covid19',
 'more than 400 companies — including tesla pfizer delta air lines and amazon — have signed on to suppo civil rights legislation for lgbtq people that is moving through congress advocates said tuesday nbcout',
 'a delta 

In [19]:
# from: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python/49146722#49146722
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [20]:
media_text = [remove_emojis(text) for text in media_text] # remove emojis

For BERTopic removing stopwords as a preprocessing step is not advised as the transformer-based embedding models it uses need the full context in order to create accurate embeddings. (https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html)

Hence, finish with data-cleaning and save the cleaned text.

In [21]:
media_min["text_clean"] = media_text
media_min.head()

,created_at,text,id,author_id,lang,username,text_clean
0,2021-12-31 12:06:05,'That was not the basis of the CDC's decision'...,1476887381336801280,2836421,en,MSNBC,that was not the basis of the cdcs decision dr...
1,2021-12-30 02:46:02,'That was not the basis of the CDC's decision'...,1476384051154112512,2836421,en,MSNBC,that was not the basis of the cdcs decision dr...
2,2021-12-29 13:31:07,'That was not the basis of the CDC's decision'...,1476184003526938624,2836421,en,MSNBC,that was not the basis of the cdcs decision dr...
3,2021-12-29 06:31:02,'That was not the basis of the CDC's decision'...,1476078288661581824,2836421,en,MSNBC,that was not the basis of the cdcs decision dr...
4,2021-08-25 19:34:01,Delta Air Lines CEO tells employees that they ...,1430614459077349376,2836421,en,MSNBC,delta air lines ceo tells employees that they ...


In [22]:
media_min.to_csv("media_clean.csv", sep = ",")

### Preprocess the corporate dataset

In [23]:
corporation = pd.read_json('corporation.jsonl')

In [24]:
corporation.head()

,context_annotations,source,entities,author_id,lang,possibly_sensitive,text,attachments,conversation_id,created_at,id,public_metrics,reply_settings,author,__twarc,referenced_tweets,geo
0,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 115, 'end': 138, 'url': 'h...",378197959,en,False,Hitting the road? Here’s how 3M is innovating ...,"{'media_keys': ['13_1476609856396316673'], 'me...",1476609915536003072,2021-12-30 17:43:32,1476609915536003072,"{'retweet_count': 4, 'reply_count': 2, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None
1,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 145, 'end': 168, 'url': 'h...",378197959,en,False,79% of people believe that communities need to...,"{'media_keys': ['13_1475496712689131530'], 'me...",1475496744720945152,2021-12-27 16:00:12,1475496744720945152,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None
2,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 223, 'end': 246, 'url': 'h...",378197959,en,False,Make the holidays merry and bright with @3MSco...,"{'media_keys': ['3_1474394454693920776'], 'med...",1474394468782678016,2021-12-24 15:00:08,1474394468782678016,"{'retweet_count': 1, 'reply_count': 2, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None
3,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 191, 'end': 214, 'url': 'h...",378197959,en,False,'Tis the season to give back to our communitie...,"{'media_keys': ['13_1474032078022975494'], 'me...",1474032177973276672,2021-12-23 15:00:32,1474032177973276672,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None
4,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 125, 'end': 148, 'url': 'h...",378197959,en,False,64% of the world’s population is concerned for...,"{'media_keys': ['13_1473669677071187971'], 'me...",1473669732553342976,2021-12-22 15:00:18,1473669732553342976,"{'retweet_count': 3, 'reply_count': 2, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None


In [25]:
len(corporation)

166078

In case any tweets are retrieved more than once, drop duplicates based on the tweet id

In [26]:
corporation = corporation.drop_duplicates(subset=['id'], ignore_index = True)

In [27]:
len(corporation)

166076

Add name of the authors to the dataframe

In [28]:
c_author = pd.json_normalize(corporation["author"])
c_author.head()

,verified,description,protected,url,created_at,location,profile_image_url,name,id,username,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,entities.url.urls,entities.description.hashtags,entities.description.urls,pinned_tweet_id,entities.description.mentions,entities.description.cashtags
0,True,"Here, we innovate with purpose & use #science ...",False,https://t.co/z42xFblwhd,2011-09-22T20:12:19.000Z,"St Paul, MN",https://pbs.twimg.com/profile_images/723375867...,3M,378197959,3M,1373299,4849,12788,1236,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 37, 'end': 45, 'tag': 'science'}, {...",NaN,NaN,NaN,NaN
1,True,"Here, we innovate with purpose & use #science ...",False,https://t.co/z42xFblwhd,2011-09-22T20:12:19.000Z,"St Paul, MN",https://pbs.twimg.com/profile_images/723375867...,3M,378197959,3M,1373299,4849,12788,1236,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 37, 'end': 45, 'tag': 'science'}, {...",NaN,NaN,NaN,NaN
2,True,"Here, we innovate with purpose & use #science ...",False,https://t.co/z42xFblwhd,2011-09-22T20:12:19.000Z,"St Paul, MN",https://pbs.twimg.com/profile_images/723375867...,3M,378197959,3M,1373299,4849,12788,1236,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 37, 'end': 45, 'tag': 'science'}, {...",NaN,NaN,NaN,NaN
3,True,"Here, we innovate with purpose & use #science ...",False,https://t.co/z42xFblwhd,2011-09-22T20:12:19.000Z,"St Paul, MN",https://pbs.twimg.com/profile_images/723375867...,3M,378197959,3M,1373299,4849,12788,1236,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 37, 'end': 45, 'tag': 'science'}, {...",NaN,NaN,NaN,NaN
4,True,"Here, we innovate with purpose & use #science ...",False,https://t.co/z42xFblwhd,2011-09-22T20:12:19.000Z,"St Paul, MN",https://pbs.twimg.com/profile_images/723375867...,3M,378197959,3M,1373299,4849,12788,1236,"[{'start': 0, 'end': 23, 'url': 'https://t.co/...","[{'start': 37, 'end': 45, 'tag': 'science'}, {...",NaN,NaN,NaN,NaN


In [29]:
corporation = corporation.join(c_author["username"])

In [30]:
corporation.head()

,context_annotations,source,entities,author_id,lang,possibly_sensitive,text,attachments,conversation_id,created_at,id,public_metrics,reply_settings,author,__twarc,referenced_tweets,geo,username
0,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 115, 'end': 138, 'url': 'h...",378197959,en,False,Hitting the road? Here’s how 3M is innovating ...,"{'media_keys': ['13_1476609856396316673'], 'me...",1476609915536003072,2021-12-30 17:43:32,1476609915536003072,"{'retweet_count': 4, 'reply_count': 2, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,3M
1,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 145, 'end': 168, 'url': 'h...",378197959,en,False,79% of people believe that communities need to...,"{'media_keys': ['13_1475496712689131530'], 'me...",1475496744720945152,2021-12-27 16:00:12,1475496744720945152,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,3M
2,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 223, 'end': 246, 'url': 'h...",378197959,en,False,Make the holidays merry and bright with @3MSco...,"{'media_keys': ['3_1474394454693920776'], 'med...",1474394468782678016,2021-12-24 15:00:08,1474394468782678016,"{'retweet_count': 1, 'reply_count': 2, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,3M
3,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 191, 'end': 214, 'url': 'h...",378197959,en,False,'Tis the season to give back to our communitie...,"{'media_keys': ['13_1474032078022975494'], 'me...",1474032177973276672,2021-12-23 15:00:32,1474032177973276672,"{'retweet_count': 3, 'reply_count': 1, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,3M
4,"[{'domain': {'id': '45', 'name': 'Brand Vertic...",Sprinklr Publishing,"{'urls': [{'start': 125, 'end': 148, 'url': 'h...",378197959,en,False,64% of the world’s population is concerned for...,"{'media_keys': ['13_1473669677071187971'], 'me...",1473669732553342976,2021-12-22 15:00:18,1473669732553342976,"{'retweet_count': 3, 'reply_count': 2, 'like_c...",everyone,"{'verified': True, 'description': 'Here, we in...",{'url': 'https://api.twitter.com/2/tweets/sear...,None,None,3M


Minimize the dataset, keep only the columns needed for the study

In [31]:
cor_min = corporation[['created_at','text','id', 'author_id', 'lang', 'username']]
cor_min.head()

,created_at,text,id,author_id,lang,username
0,2021-12-30 17:43:32,Hitting the road? Here’s how 3M is innovating ...,1476609915536003072,378197959,en,3M
1,2021-12-27 16:00:12,79% of people believe that communities need to...,1475496744720945152,378197959,en,3M
2,2021-12-24 15:00:08,Make the holidays merry and bright with @3MSco...,1474394468782678016,378197959,en,3M
3,2021-12-23 15:00:32,'Tis the season to give back to our communitie...,1474032177973276672,378197959,en,3M
4,2021-12-22 15:00:18,64% of the world’s population is concerned for...,1473669732553342976,378197959,en,3M


Check missing values

In [32]:
cor_min.isna().sum()

created_at    0
text          0
id            0
author_id     0
lang          0
username      0
dtype: int64

Check language, to make it easier for human interpretation, only keep the English tweets

In [33]:
cor_min["lang"].value_counts()

en     161135
es       1254
und       941
fr        723
de        436
ar        232
it        209
zh        147
pt        114
th        111
tr        111
ja        106
in        102
nl         95
pl         61
sl         32
ru         30
sv         27
vi         20
ca         20
ro         19
fi         16
tl         15
no         14
hi         13
iw         12
da         12
cy         11
et         10
ht          9
ko          7
bn          7
cs          6
kn          6
lt          5
el          3
eu          1
hu          1
is          1
gu          1
mr          1
Name: lang, dtype: int64

In [34]:
cor_min = cor_min[cor_min['lang']=='en']

In [35]:
len(cor_min)

161135

Some typical clean-up steps

In [36]:
cor_text = cor_min["text"].to_list()

In [37]:
cor_text[:20]

['Hitting the road? Here’s how 3M is innovating to get you to and from your destination safely this holiday season. \nhttps://t.co/kmKxiOBq1r https://t.co/zxLcZGWGO8',
 '79% of people believe that communities need to work harder to help cars, bicycles, and pedestrians share the road. See how 3M is doing its part. https://t.co/kOOPaRocF8 https://t.co/UfYPQgT9FA',
 'Make the holidays merry and bright with @3MScotchlite Reflective Material. 🌟 3M Science behind retroreflection will keep you and your pet safe and visible at night! #3mreflective #scotchlite\n\n📷 Instagram: freya.thedoodle 🐶 https://t.co/CmDCMtbnvL',
 '\'Tis the season to give back to our communities. See how @SCGSigns brought holiday spirit by wrapping these buses with @3mfilms for @cityofnorwalkca\'s annual "Stuff-A-Bus" charity drive. 🚌🎄🎅 https://t.co/1B39aFAd42',
 '64% of the world’s population is concerned for their safety during holiday travel. Learn how 3M is working to keep you safe: https://t.co/G1eNslQpgk https://t.

In [38]:
cor_text = [re.sub(r'https?://\S+', '', text) for text in cor_text] #remove url
cor_text = ["".join([l for l in text if l not in punctuation]) for text in cor_text]  #remove punctuation
cor_text = [text.lower() for text in cor_text]  # convert to lower case
cor_text = [text.replace ("rt", "") for text in cor_text] # remove the word "rt"
cor_text = [" ".join(text.split()) for text in cor_text]   # remove double spaces by splitting the strings into words and joining these words again

In [39]:
cor_text = [remove_emojis(text) for text in cor_text] # remove emojis

In [40]:
cor_text[:10]

['hitting the road here’s how 3m is innovating to get you to and from your destination safely this holiday season',
 '79 of people believe that communities need to work harder to help cars bicycles and pedestrians share the road see how 3m is doing its pa',
 'make the holidays merry and bright with 3mscotchlite reflective material  3m science behind retroreflection will keep you and your pet safe and visible at night 3mreflective scotchlite  instagram freyathedoodle ',
 'tis the season to give back to our communities see how scgsigns brought holiday spirit by wrapping these buses with 3mfilms for cityofnorwalkcas annual stuffabus charity drive ',
 '64 of the world’s population is concerned for their safety during holiday travel learn how 3m is working to keep you safe',
 'reusing wrapping paper is a great way to make the holidays more sustainable you can even make your own bows out of smaller scraps share this tip with your friends and families ',
 'did you know your scotchbrite scrub 

Finish with data-cleaning and save the cleaned text.

In [41]:
cor_min["text_clean"] = cor_text
cor_min.head()

,created_at,text,id,author_id,lang,username,text_clean
0,2021-12-30 17:43:32,Hitting the road? Here’s how 3M is innovating ...,1476609915536003072,378197959,en,3M,hitting the road here’s how 3m is innovating t...
1,2021-12-27 16:00:12,79% of people believe that communities need to...,1475496744720945152,378197959,en,3M,79 of people believe that communities need to ...
2,2021-12-24 15:00:08,Make the holidays merry and bright with @3MSco...,1474394468782678016,378197959,en,3M,make the holidays merry and bright with 3mscot...
3,2021-12-23 15:00:32,'Tis the season to give back to our communitie...,1474032177973276672,378197959,en,3M,tis the season to give back to our communities...
4,2021-12-22 15:00:18,64% of the world’s population is concerned for...,1473669732553342976,378197959,en,3M,64 of the world’s population is concerned for ...


In [42]:
cor_min.to_csv("corporation_clean.csv", sep = ",")